In [ ]:
import re
import parse
import pdfplumber
import pandas as pd
from collections import namedtuple
from PyPDF2 import PdfFileWriter, PdfFileReader
import textwrap
import os

In [ ]:
import os
list = os.listdir('./final')

### Removing the unneeded pages 

In [ ]:
import textwrap
rmSp = re.compile(r'^\s+',re.MULTILINE)


def Pages_To_Keep(filename,dictionnaire,newpath):
    output = PdfFileWriter()
    lines = []
    pagesIndex = []
    i=0
    with pdfplumber.open(filename) as pdf:
        pages = pdf.pages
        for page in pdf.pages:
            text = page.extract_text()
            if(text is not None) and (i<23):
                for line in text.split('\n'):
                    if rmSp.match(line):
                        line=textwrap.dedent(line)
                    if line.lower().startswith('etat  de résultat technique de') or line.startswith('Conclusion') or line.startswith('Note') or line.startswith('NOTES') or line.startswith('NOTE') or line.upper().startswith('ETAT DE RÉSULTAT TECHNIQUE') or line.lower().startswith('pages') or line.startswith('AC11') or line.startswith('BILAN - ACTIF ') or line.startswith('BILAN – CAPITAUX') or line.lower().startswith('sommaire') or line.lower().startswith('table des ') or line.startswith('Rapport des commissaires aux comptes') or line.startswith('R3- VARIATIONS DES STOCKS') or line.startswith('Soldes Intermédiaires de Gestion') or line.startswith('I N D E X') or line.startswith('SCHEMA DES SOLDES') or line.startswith('Bilan - actifs :') or line.startswith('Rubrique') or line.startswith('Les reclassements suivants') or line.startswith('F- NOTES RELATIVES') or line.startswith('AC71 Avoirs en banque, CCP et Caisse :') or line.startswith('S O M M A I R E') or line.find('NOTES RELATIVES AU BILAN')!= -1 or line.startswith('P L A N') or line.startswith('F1- Amortissements et provisions') or line.startswith('F10- Encaissements provenant des') or line.lower().startswith('etat de resultat technique ') or line.lower().startswith('3-3-5-') or line.lower().startswith('5.3') or line.startswith('II-') :
                        break
                    if (line.startswith(tuple(dictionnaire)) and (i not in pagesIndex)):
                        pagesIndex.append(i)               
                        break
                i=i+1
        infile = PdfFileReader(filename, 'rb')

        for i in pagesIndex:
            p = infile.getPage(i)
            output.addPage(p)

        with open(newpath+filename, 'wb') as f:
            output.write(f)




In [ ]:
with open ("dictionnaire.txt", "r",encoding='utf-8') as f:
    data=f.read()
    dictionnaire=data.split('\n')


In [ ]:
newpath ="shortPDF/"
if not os.path.exists(newpath):
    os.makedirs(newpath)
for l in list:    
    Pages_To_Keep(l,dictionnaire,newpath)


# Phase 2 : Data Preparation
```

```

### Importing modules 


## Data Extraction 

```

```




In [ ]:
liste = os.listdir('./shortPDF')
liste

['ab-etats-financiers-annuels-31-12-2017.pdf',
 'ab-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'ab-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'adwya-etats-financiers-annuels-31-12-2016.pdf',
 'adwya-etats-financiers-annuels-31-12-2018.pdf',
 'adwya-etats-financiers-annuels-31-12-2019.pdf',
 'aetec-etats-financiers-annuels-31-12-2018.pdf',
 'alkim-etats-financiers-annuels-31-12-2018.pdf',
 'alkim-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'alkim-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'ams-etats-financiers-annuels-consolides-31-12-2015.pdf',
 'artes-etats-financiers-annuels-31-12-2018.pdf',
 'artes-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'artes-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'atb-etats-financiers-annuels-31-12-2017.pdf',
 'atb-etats-financiers-annuels-31-12-2018.pdf',
 'atb-etats-financiers-annuels-31-12-2019.pdf',
 'bt-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'cc-etats-financiers-annuels-31

In [ ]:
def extract_value2(str):
    fin=str.rfind('  ',0,len(str)-4)
    debut=str.rfind('  ',0,fin-2)
    
    ch=str[debut+2:fin].replace('  ',' ')
    verif=re.compile(r'(([\(]?[a-z,A-Z, ,ô,è,é,\',:,à,É,.,ê,’,*,À,&][\)]?)|([\(]?[-][a-z,A-Z,ô,è,é,\',:,à,É,.,ê,’,*,À,&][\)]?))*(.*)')
    valeur=verif.search(ch).group(4)
    if(debut==-1 and fin==-1 and valeur!='' ):
        valeur=get_val(str)
        if(valeur=="0"):
            return valeur
        #verifier si la premiere année commence par le parenthése
        prenthese=re.compile(r'(^[\(][0-9, ]+[\)])(.*)')
        #vérifier si l'année précedente avec parenthese
        prenthese2=re.compile(r'(.*)([\(][0-9, ]+[\)])(.*)')
        #méme que les parentheses mais avec <>
        prenthese3=re.compile(r'(^[<][0-9, ]+[>])(.*)')
        prenthese4=re.compile(r'(.*)([<][0-9, ]+[>])(.*)')
        if(prenthese.search(valeur)):
            val=prenthese.search(valeur).group(1)
        elif(prenthese2.search(valeur)):
            val=prenthese2.search(valeur).group(1)
        elif(prenthese3.search(valeur)):
            val=prenthese3.search(valeur).group(1)
        elif(prenthese4.search(valeur)):
            val=prenthese4.search(valeur).group(1)
    
        
        else:
            #longueur de la valeur de l'année précedente n'est pas multiple de 3
            extrait=re.compile(r'(.+) [-]?([1-9]|[1-9][0-9]) (.+)')
            if(extrait.search(valeur)):
                val=extrait.search(valeur).group(1)
            else:
                if valeur[0]==' ' :
                    valeur=valeur[1:len(valeur)]
                print("erreur:", valeur)
                while valeur[len(valeur)-1]==' ':
                    valeur=valeur[0:len(valeur)-1]
                if(re.compile(r'(^[-]?[1-9][0-9][0-9] )(.+)').search(valeur)):
                    print("val23:",valeur)
                    c=int(valeur.count(' ')  // 2)
                    fin=(3*(c+1))+c
                    val=valeur[0:fin+1]
                    print("val erreur:",val)
                else :
                    print("ici")
                    extrait=re.compile(r'([1-9] |[1-9][0-9] )(.+)')
                    extract=extrait.search(valeur).group(2)
                    c=int(extract.count(' ')  // 2)
                    fin=(3*(c+1))+c

                    chaine=extract[0:fin+1]
                    val=extrait.search(valeur).group(1)+chaine
        ch=val
    return ch

In [ ]:
def get_val(valeur):
    #verifier d'abord si la valeur est égale à 0
    verif3=re.compile(r'(--)|(-\((.*)\))')
    #verifier les formes particuliéres
    verif=re.compile(r'(([\(]?(([a-z,A-Z][.,-]?[1-9][1-9]?)|[1-9][.,-][0-9])[\)]?)|[\(][0-9][0-9]?[\)])(.*)')
    #verifier s'il existe un ou 2 entiers avant le signe -
    verif2=re.compile(r'( [1-9][0-9]?) (-[ ]?(.*))')
    if(verif3.search(valeur)):
        valeur="0"
    elif(verif.search(valeur)):
        valeur=verif.search(valeur).group(5)
    elif(verif2.search(valeur)):
        valeur=verif2.search(valeur).group(2)
    else:
        verif=re.compile(r'(([\(]?[a-z,A-Z, ,ô,è,é,\',:,à,É,.,ê,’,*,À,&][\)]?)|([\(]?[-][a-z,A-Z,ô,è,é,\',:,à,É,.,ê,’,*,À,&][\)]?))*(.*)')
        valeur=verif.search(valeur).group(4)
        verif=re.compile(r'^([1-9][1-9]|[1-9]) ([\(][0-9, ]+[\)](.*))')
        verif2=re.compile(r'^([1-9][1-9]|[1-9]) ([<][0-9, ]+[>](.*))')
        if(verif.search(valeur)):
            valeur=verif.search(valeur).group(2)
        elif(verif2.search(valeur)):
            valeur=verif2.search(valeur).group(2)
        else:
            verif=re.compile(r'^([1-9][1-9]|[-]?[1-9]) ([-]?([1-9][1-9]|[1-9]) (.*))')
            if(verif.search(valeur)):
                valeur=verif.search(valeur).group(2)
            else:
                verif=re.compile(r'(([\(]?[a-z,A-Z, ,ô,è,é,\',:,à,É,.,ê,’,*,À,&][\)]?)|([\(]?[-][a-z,A-Z,ô,è,é,\',:,à,É,.,ê,’,*,À,&][\)]?))*(.*)')
                valeur=verif.search(valeur).group(4)
                
    return valeur

In [ ]:
def no_double_espace(valeur):
    
    if(valeur=="0"):
        return valeur
    #verifier si la premiere année commence par le parenthése
    prenthese=re.compile(r'(^[\(][0-9, ]+[\)])(.*)')
    #vérifier si l'année précedente avec parenthese
    prenthese2=re.compile(r'(.*)([\(][0-9, ]+[\)])(.*)')
    #méme que les parentheses mais avec <>
    prenthese3=re.compile(r'(^[<][0-9, ]+[>])(.*)')
    prenthese4=re.compile(r'(.*)([<][0-9, ]+[>])(.*)')
    if(prenthese.search(valeur)):
        val=prenthese.search(valeur).group(1)
    elif(prenthese2.search(valeur)):
        val=prenthese2.search(valeur).group(1)
    elif(prenthese3.search(valeur)):
        val=prenthese3.search(valeur).group(1)
    elif(prenthese4.search(valeur)):
        val=prenthese4.search(valeur).group(1)
    

    else:
        #longueur de la valeur de l'année précedente n'est pas multiple de 3
        extrait=re.compile(r'(.+) [-]?([1-9]|[1-9][0-9]) (.+)')
        if(extrait.search(valeur)):
            val=extrait.search(valeur).group(1)
        else:
            if valeur[0]==' ' :
                valeur=valeur[1:len(valeur)]
            while valeur[len(valeur)-1]==' ':
                valeur=valeur[0:len(valeur)-1]
            if(re.compile(r'(^[-]?[1-9][0-9][0-9] )(.+)').search(valeur)):
                c=int(valeur.count(' ')  // 2)
                fin=(3*(c+1))+c
                val=valeur[0:fin+1]
            else :
                extrait=re.compile(r'([1-9] |[1-9][0-9] )(.+)')
                extract=extrait.search(valeur).group(2)
                c=int(extract.count(' ')  // 2)
                fin=(3*(c+1))+c

                chaine=extract[0:fin+1]

                val=extrait.search(valeur).group(1)+chaine
    return val;

    

In [ ]:
def extract_value(ch):
    print("input:",ch)
    while(ch.rfind('   ')!=-1):
        ch=ch.replace('   ','  ')
    vir=ch.count(',')

    ch=ch.replace(',', ' ')
    while(ch.rfind('- ')!=-1):
        ch=ch.replace('- ','-')
    
    #Eliminer tous les caractéres
    verif=re.compile(r'(([\(]?[a-z,A-Z, ,ô,è,é,\',:,à,É,ê,’,*,À,&][\)]?)|([\(]?[-][a-z,A-Z,ô,è,é,\',:,à,É,ê,’,*,À,&][\)]?))*(.*)')  
    valeur=verif.search(ch).group(4)
    print("valeur: ",valeur)
    if(valeur=='')|(valeur==' '):
        val=''
    elif(valeur.rfind('  ')!=-1):
        

        val=extract_value2(ch)
        print("ici2:",val)
        #print("ap manel:", val)
        if(val=="-"):
            val="0"
            return val
        if(len(val)<=3):
            ch=ch[len(ch)::-1]
            ch=ch.replace('  ',' ',1)
            ch=ch[len(ch)::-1]
            print("ici3",ch)
            val=extract_value2(ch)
        val=get_val(val)
        print("ici4",val)
        verif=re.compile(r'(([\(]?[a-z,A-Z, ,ô,è,é,\',:,à,É,.,ê,’,*,À,&][\)]?)|([\(]?[-][a-z,A-Z,ô,è,é,\',:,à,É,.,ê,’,*,À,&][\)]?))*(.*)')
        if(verif.search(val).group(1)!=None):
            
            verif=re.compile(r'(([\(]?[a-z,A-Z, ,ô,è,é,\',:,à,É,.,ê,’,*,À,&][\)]?)|([\(]?[-][a-z,A-Z,ô,è,é,\',:,à,É,ê,.,’,*,À,&][\)]?))*(.*)')
            val=verif.search(val).group(4)
        #double espace aprés le premier chiffre de la valeur du de la dérniere année
        verif_extra_int_end=re.compile(r'((.*)[0-9][0-9][0-9]) [0-9]?[0-9]$')
        if(verif_extra_int_end.search(val)):
            val=verif_extra_int_end.search(val).group(1)
        verif_espace=re.compile(r'(^[0-9][0-9]?  )(.+)')
        if(len(val)<3)and(verif_espace.search(valeur)):
            val=no_double_espace(verif_espace.search(valeur).group(2))
            
       
    else:
        valeur=get_val(ch)
        if(valeur=="0"):
            return valeur
        #verifier si la premiere année commence par le parenthése
        prenthese=re.compile(r'(^[\(][0-9, ]+[\)])(.*)')
        #vérifier si l'année précedente avec parenthese
        prenthese2=re.compile(r'(.*)([\(][0-9, ]+[\)])(.*)')
        #méme que les parentheses mais avec <>
        prenthese3=re.compile(r'(^[<][0-9, ]+[>])(.*)')
        prenthese4=re.compile(r'(.*)([<][0-9, ]+[>])(.*)')
        if(prenthese.search(valeur)):
            val=prenthese.search(valeur).group(1)
        elif(prenthese2.search(valeur)):
            val=prenthese2.search(valeur).group(1)
        elif(prenthese3.search(valeur)):
            val=prenthese3.search(valeur).group(1)
        elif(prenthese4.search(valeur)):
            val=prenthese4.search(valeur).group(1)
    

        else:
            #longueur de la valeur de l'année précedente n'est pas multiple de 3
            extrait=re.compile(r'(.+) [-]?([1-9]|[1-9][0-9]) (.+)')
            if(extrait.search(valeur)):
                val=extrait.search(valeur).group(1)
            else:
                if valeur[0]==' ' :
                    valeur=valeur[1:len(valeur)]
                while valeur[len(valeur)-1]==' ':
                    valeur=valeur[0:len(valeur)-1]
                if(re.compile(r'(^[-]?[1-9][0-9][0-9] )(.+)').search(valeur)):
                    c=int(valeur.count(' ')  // 2)
                    fin=(3*(c+1))+c
                    val=valeur[0:fin+1]
                else :
                    extrait=re.compile(r'([1-9] |[1-9][0-9] )(.+)')
                    extract=extrait.search(valeur).group(2)
                    c=int(extract.count(' ')  // 2)
                    fin=(3*(c+1))+c

                    chaine=extract[0:fin+1]

                    val=extrait.search(valeur).group(1)+chaine
    
    if vir==2 :
        if(val[len(val)-1]==(")")):
            val=val[0:len(val)-4] 
            val=val+')'
        else :
            val=val[0:len(val)-3]
            
    return val 

In [ ]:
def extract_data_from_bilan(textbilan,dict_bilan):

    TCPP = ''
    CP = ''
    PNC = ''
    TA = ''
    TP = ''
    PC = ''
    RES = ''
    CAP = ''
    ANC = ''
    AC = ''

    for line in textbilan:
        if ((all(x in line.lower() for x in dict_bilan[0].split(' ')) or all(x in line.lower() for x in dict_bilan[1].split(' '))) 
            and ("Notes" not in line)):
            TCPP=extract_value(line) 

        elif(( (all(x in line.lower() for x in dict_bilan[2].split(' ')) and ("et" not in line)) or 
            all(x in line.lower() for x in ['cx','propres']))):

            CP=extract_value(line)            
    
            if (line[0:38] == dict_bilan[13] or line[0:46] == dict_bilan[12]):
                CP=''

        elif all(x in line.lower() for x in dict_bilan[3].split(' ')):
            PNC=extract_value(line)

        elif all(x in line.lower() for x in dict_bilan[4].split(' ')):
            PC=extract_value(line)

        elif ((all(x in line.lower() for x in dict_bilan[5].split(' ')) or all(x in line.lower() for x in dict_bilan[6].split(' ')))
              and ("," not in line)) :
            TP=extract_value(line)

        elif all(x in line.lower() for x in dict_bilan[7].split(' ')):
            ANC=extract_value(line)

        elif all(x in line.lower() for x in dict_bilan[8].split(' ')):
            AC=extract_value(line)

            
        elif (all(x in line.lower() for x in dict_bilan[9].split(' ')) or all(x in line.lower() for x in dict_bilan[10].split(' '))):
            TA=extract_value(line)  
            if(len(TA)>3 and (TA[-2]==' ')):
                TA=TA[0:len(TA)-2]
         
        elif any(x in line.lower() for x in dict_bilan[14].split(' ')):
            if(RES==''):
                print("icii",line)
                RES=extract_value(line)
            
          #  print('reserve ',RES)
        elif any(x in line.lower() for x in dict_bilan[15].split(' ')):
            CAP=extract_value(line)
          #  print(line)
            
    return [TCPP,CP,PNC,TA,TP,PC,ANC,AC,RES,CAP]



In [ ]:
def extract_data_from_etat_resultat(textetat,dict_etat_resultat):

   
    TProdExpl =''
    TChargeExpl =''
    R_Expl =''
    R_Exercice =''
    ActOrd ='' 

    for line in textetat:
        if ((any(x in line.lower() for x in dict_etat_resultat[0].split(' '))) and 
              (any(x in line.lower() for x in dict_etat_resultat[2].split(' '))) and ("placements" not in line)):
            
            TProdExpl=extract_value(line)

        elif ((any(x in line.lower() for x in dict_etat_resultat[1].split(' '))) and 
              (any(x in line.lower() for x in dict_etat_resultat[2].split(' ')))):
              
            TChargeExpl=extract_value(line)
 

        elif ((any(x in line.lower() for x in dict_etat_resultat[3].split(' '))) and 
              (any(x in line.lower() for x in dict_etat_resultat[4].split(' ')))):
            R_Expl=extract_value(line)


        elif ((any(x in line.lower() for x in dict_etat_resultat[3].split(' '))) and 
              (any(x in line.lower() for x in dict_etat_resultat[5].split(' ')))):
             
            R_Exercice=extract_value(line)

        elif ((any(x in line.lower() for x in dict_etat_resultat[6].split(' '))) and 
              (any(x in line.lower() for x in dict_etat_resultat[7].split(' ')))):
            ActOrd=extract_value(line)


    return [TProdExpl,TChargeExpl,R_Expl,R_Exercice,ActOrd ]
        

In [ ]:
def extract_data_from_flux_tresorerie(texttresorerie,dict_flux_tresorerie):
    
   
    FT_Exp =''
    FT_Fin =''
    FT_Inv =''
    Variation =''
    TDeb ='' 
    TFin =''

    for line in texttresorerie:
        if ((any(x in line.lower() for x in dict_flux_tresorerie[0].split(' '))) and 
              (any(x in line.lower() for x in dict_flux_tresorerie[1].split(' ')))):
            FT_Exp=extract_value(line)    

        elif ((any(x in line.lower() for x in dict_flux_tresorerie[0].split(' '))) and 
              (any(x in line.lower() for x in dict_flux_tresorerie[2].split(' ')))):
            FT_Inv=extract_value(line)
 
        elif((any(x in line.lower() for x in dict_flux_tresorerie[0].split(' '))) and 
              (any(x in line.lower() for x in dict_flux_tresorerie[3].split(' ')))):
            FT_Fin=extract_value(line) 

        elif ((any(x in line.lower() for x in dict_flux_tresorerie[4].split(' '))) and 
              (any(x in line.lower() for x in dict_flux_tresorerie[5].split(' ')))):
            TDeb=extract_value(line)

        elif ((any(x in line.lower() for x in dict_flux_tresorerie[4].split(' '))) and 
              (any(x in line.lower() for x in dict_flux_tresorerie[6].split(' ')))):
            TFin=extract_value(line)


        elif ((any(x in line.lower() for x in dict_flux_tresorerie[7].split(' '))) and 
              (any(x in line.lower() for x in dict_flux_tresorerie[8].split(' ')))):
            Variation=extract_value(line)

    return [FT_Exp,FT_Inv,FT_Fin,TDeb,TFin,Variation]


In [ ]:
def exception_Bilan(line,dict_etat_resultat):

    R_ExerciceTemp =''
    ActOrdTemp ='' 


    if ((any(x in line.lower() for x in dict_etat_resultat[3].split(' '))) and 
        (any(x in line.lower() for x in dict_etat_resultat[5].split(' ')))):
             
        R_ExerciceTemp=extract_value(line)

    if ((any(x in line.lower() for x in dict_etat_resultat[6].split(' '))) and 
        (any(x in line.lower() for x in dict_etat_resultat[7].split(' ')))):
        ActOrdTemp=extract_value(line)

    return [R_ExerciceTemp,ActOrdTemp ]
        

In [ ]:

with open ("dict_bilan.txt", "r",encoding='utf-8') as f1:
    dict_bilan = f1.read().split('\n')

with open ("dict_etat_resultat.txt", "r",encoding='utf-8') as f2:
    dict_etat_resultat = f2.read().split('\n')
    
with open ("dict_flux_tresorerie.txt", "r",encoding='utf-8') as f3:
    dict_flux_tresorerie = f3.read().split('\n')

ligne = namedtuple('ligne', 'Entreprise Annee TCPP CapitalPropre PassifNonC TActif TPassif PassifCourant ActifNonC ActifCourant Reserves CapitalSociale TProdExploitation TChargeExploitation R_Exploitation Res_Exsercice ActiviteOrdinaire FT_Exploitation FT_Investisement FT_Financement TDebut TFin Variation')



In [ ]:
lines = []
rmSp = re.compile(r'^\s+',re.MULTILINE)
for l in liste[0:80] :
    s=l[0:(len(l)-4)].split('-')
    Entreprise=s[0]
    print(Entreprise)
    Annee=s[-1]
    textbilan = []
    textetat = []
    texttresorerie = []
    tempResExercice=""
    tempActOrd=""
    with pdfplumber.open('shortPDF/'+l) as pdf:
        pages = pdf.pages
        #print(l)
        if(len(pages) == 3):
          
      
            for line in pages[0].extract_text().split('\n'):
                if rmSp.match(line):
                    line=textwrap.dedent(line)
                if line.lower().startswith(('total','capitaux','t o ta  l','actif ','passif ','sous-total','réserve','cp','reserves')): 
                    textbilan.append(line)
                    print(line)
                if (line.lower().startswith('résultat') or line.lower().startswith('resultat') or line.lower().startswith('resultats')) :
                    temp=exception_Bilan(line,dict_etat_resultat)
                    tempResExercice=temp[0]
                    tempActOrd=temp[1]

            
            for line in pages[1].extract_text().split('\n'):
                if rmSp.match(line):
                    line=textwrap.dedent(line)
                if line.lower().startswith(('  résultat','résultat','total','charges','produit','t o tal','resultat','resultats')): 
                    textetat.append(line)
            
            for line in pages[2].extract_text().split('\n'):
                if rmSp.match(line):
                    line=textwrap.dedent(line)
                if line.lower().startswith(('flux','variation','liquid', 'f l','   trésorerie','total','trésorerie','fluflux','vavariation')): 
                    texttresorerie.append(line)
           
                    
            l1=tuple(extract_data_from_bilan(textbilan,dict_bilan))
            l2=(extract_data_from_etat_resultat(textetat,dict_etat_resultat))
            l3=(extract_data_from_flux_tresorerie(texttresorerie,dict_flux_tresorerie))
            
            if((l2[3]=="") and tempResExercice != ""):
                l2[3]=tempResExercice
                    
            if((l2[3]=="") and tempResExercice != ""):
                l2[3]=tempResExercice
                
            lines.append(ligne(Entreprise,Annee,l1[0],l1[1],l1[2],l1[3],l1[4],l1[5],l1[6],l1[7],l1[8],l1[9],l2[0],l2[1],l2[2],l2[3],l2[4],
                          l3[0],l3[1],l3[2],l3[3],l3[4],l3[5]))
        
        elif(len(pages) == 4):
    
            for line in pages[0].extract_text().split('\n'):
                if rmSp.match(line):
                    line=textwrap.dedent(line)

                if line.lower().startswith(('total','t o ta  l','actif ','réserve')): 
                    textbilan.append(line)
    
                    
            for line in pages[1].extract_text().split('\n'):
                if rmSp.match(line):
                    line=textwrap.dedent(line)
                if line.lower().startswith(('total','capitaux','t o ta  l','passif ','réserve','cp','reserves','capital')): 
                    textbilan.append(line)
                print(line)
                   
                if (line.lower().startswith('résultat') or line.lower().startswith('resultat') or line.lower().startswith('resultats')) :
                    temp=exception_Bilan(line,dict_etat_resultat)
                    tempResExercice=temp[0]
                    tempActOrd=temp[1]

                    
            for line in pages[2].extract_text().split('\n'):
                print(line)
                if rmSp.match(line):
                    line=textwrap.dedent(line)
                if line.lower().startswith(('résultat','total','charges','produit','t o tal','resultat','  résultat')): 
                    textetat.append(line)

            for line in pages[3].extract_text().split('\n'):
                
         
                if rmSp.match(line):
                    line=textwrap.dedent(line)
                    
                if line.lower().startswith(('flux','variation','liquid', 'f l','   trésorerie','total','trésorerie','fluflux','vavariation','- trésorerie')): 
                    texttresorerie.append(line)
            
                   
                
            l1=(extract_data_from_bilan(textbilan,dict_bilan))
            l2=(extract_data_from_etat_resultat(textetat,dict_etat_resultat))
            l3=(extract_data_from_flux_tresorerie(texttresorerie,dict_flux_tresorerie))
        
            if((l2[3]=="") and tempResExercice != ""):
                l2[3]=tempResExercice
                    
            if((l2[4]=="") and tempActOrd != ""):
                l2[4]=tempResExercice
                
            lines.append(ligne(Entreprise,Annee,l1[0],l1[1],l1[2],l1[3],l1[4],l1[5],l1[6],l1[7],l1[8],l1[9],l2[0],l2[1],l2[2],l2[3],l2[4],
                          l3[0],l3[1],l3[2],l3[3],l3[4],l3[5]))


ab
Total actifs  8 656 198  8 242 917 
Total passifs  7 837 241  7 508 586 
CP1  Capital     132 405  127 313 
CP2  Réserves     572 213  516 585 
CP4  Autres capitaux propres     423  423 
CP5  Résultats reportés     5  4 
CP6  Résultat de l’exercice     113 911  90 006 
Total capitaux propres  (3)  818 957  734 331 
Total capitaux propres et passifs  8 656 198  8 242 917 
input: Total actifs  8 656 198  8 242 917 
valeur:  8 656 198  8 242 917 
ici2: 8 656 198
ici4 8 656 198
input: Total passifs  7 837 241  7 508 586 
valeur:  7 837 241  7 508 586 
ici2: 7 837 241
ici4 7 837 241
input: CP1  Capital     132 405  127 313 
valeur:  1  Capital  132 405  127 313 
ici2: 132 405
ici4 132 405
icii CP2  Réserves     572 213  516 585 
input: CP2  Réserves     572 213  516 585 
valeur:  2  Réserves  572 213  516 585 
ici2: 572 213
ici4 572 213
input: CP4  Autres capitaux propres     423  423 
valeur:  4  Autres capitaux propres  423  423 
ici2: 423
ici3 CP4  Autres capitaux propres  423 423 
ic

In [ ]:
lines = []
rmSp = re.compile(r'^\s+',re.MULTILINE)
for l in liste[0:80] :
    s=l[0:(len(l)-4)].split('-')
    Entreprise=s[0]
    print(Entreprise)
    Annee=s[-1]
    textbilan = []
    textetat = []
    texttresorerie = []
    tempResExercice=""
    tempActOrd=""
    with pdfplumber.open('shortPDF/'+l) as pdf:
        pages = pdf.pages
        #print(l)
        if(len(pages) == 3):
          
      
            for line in pages[0].extract_text().split('\n'):
                if rmSp.match(line):
                    line=textwrap.dedent(line)
                if line.lower().startswith(('total','capitaux','t o ta  l','actif ','passif ','sous-total','réserve','cp','reserves')): 
                    textbilan.append(line)
                    print(line)
                if (line.lower().startswith('résultat') or line.lower().startswith('resultat') or line.lower().startswith('resultats')) :
                    temp=exception_Bilan(line,dict_etat_resultat)
                    tempResExercice=temp[0]
                    tempActOrd=temp[1]

            
            for line in pages[1].extract_text().split('\n'):
                if rmSp.match(line):
                    line=textwrap.dedent(line)
                if line.lower().startswith(('  résultat','résultat','total','charges','produit','t o tal','resultat','resultats')): 
                    textetat.append(line)
            
            for line in pages[2].extract_text().split('\n'):
                if rmSp.match(line):
                    line=textwrap.dedent(line)
                if line.lower().startswith(('flux','variation','liquid', 'f l','   trésorerie','total','trésorerie','fluflux','vavariation')): 
                    texttresorerie.append(line)
           
                    
            l1=tuple(extract_data_from_bilan(textbilan,dict_bilan))
            l2=(extract_data_from_etat_resultat(textetat,dict_etat_resultat))
            l3=(extract_data_from_flux_tresorerie(texttresorerie,dict_flux_tresorerie))
            
            if((l2[3]=="") and tempResExercice != ""):
                l2[3]=tempResExercice
                    
            if((l2[3]=="") and tempResExercice != ""):
                l2[3]=tempResExercice
                
            lines.append(ligne(Entreprise,Annee,l1[0],l1[1],l1[2],l1[3],l1[4],l1[5],l1[6],l1[7],l1[8],l1[9],l2[0],l2[1],l2[2],l2[3],l2[4],
                          l3[0],l3[1],l3[2],l3[3],l3[4],l3[5]))
        
        elif(len(pages) == 4):
    
            for line in pages[0].extract_text().split('\n'):
                if rmSp.match(line):
                    line=textwrap.dedent(line)

                if line.lower().startswith(('total','t o ta  l','actif ','réserve')): 
                    textbilan.append(line)
    
                    
            for line in pages[1].extract_text().split('\n'):
                if rmSp.match(line):
                    line=textwrap.dedent(line)
                if line.lower().startswith(('total','capitaux','t o ta  l','passif ','réserve','cp','reserves','capital')): 
                    textbilan.append(line)
                print(line)
                   
                if (line.lower().startswith('résultat') or line.lower().startswith('resultat') or line.lower().startswith('resultats')) :
                    temp=exception_Bilan(line,dict_etat_resultat)
                    tempResExercice=temp[0]
                    tempActOrd=temp[1]

                    
            for line in pages[2].extract_text().split('\n'):
                print(line)
                if rmSp.match(line):
                    line=textwrap.dedent(line)
                if line.lower().startswith(('résultat','total','charges','produit','t o tal','resultat','  résultat')): 
                    textetat.append(line)

            for line in pages[3].extract_text().split('\n'):
                
         
                if rmSp.match(line):
                    line=textwrap.dedent(line)
                    
                if line.lower().startswith(('flux','variation','liquid', 'f l','   trésorerie','total','trésorerie','fluflux','vavariation','- trésorerie')): 
                    texttresorerie.append(line)
            
                   
                
            l1=(extract_data_from_bilan(textbilan,dict_bilan))
            l2=(extract_data_from_etat_resultat(textetat,dict_etat_resultat))
            l3=(extract_data_from_flux_tresorerie(texttresorerie,dict_flux_tresorerie))
        
            if((l2[3]=="") and tempResExercice != ""):
                l2[3]=tempResExercice
                    
            if((l2[4]=="") and tempActOrd != ""):
                l2[4]=tempResExercice
                
            lines.append(ligne(Entreprise,Annee,l1[0],l1[1],l1[2],l1[3],l1[4],l1[5],l1[6],l1[7],l1[8],l1[9],l2[0],l2[1],l2[2],l2[3],l2[4],
                          l3[0],l3[1],l3[2],l3[3],l3[4],l3[5]))


ab
Total actifs  8 656 198  8 242 917 
Total passifs  7 837 241  7 508 586 
CP1  Capital     132 405  127 313 
CP2  Réserves     572 213  516 585 
CP4  Autres capitaux propres     423  423 
CP5  Résultats reportés     5  4 
CP6  Résultat de l’exercice     113 911  90 006 
Total capitaux propres  (3)  818 957  734 331 
Total capitaux propres et passifs  8 656 198  8 242 917 
input: Total actifs  8 656 198  8 242 917 
valeur:  8 656 198  8 242 917 
ici2: 8 656 198
ici4 8 656 198
input: Total passifs  7 837 241  7 508 586 
valeur:  7 837 241  7 508 586 
ici2: 7 837 241
ici4 7 837 241
input: CP1  Capital     132 405  127 313 
valeur:  1  Capital  132 405  127 313 
ici2: 132 405
ici4 132 405
icii CP2  Réserves     572 213  516 585 
input: CP2  Réserves     572 213  516 585 
valeur:  2  Réserves  572 213  516 585 
ici2: 572 213
ici4 572 213
input: CP4  Autres capitaux propres     423  423 
valeur:  4  Autres capitaux propres  423  423 
ici2: 423
ici3 CP4  Autres capitaux propres  423 423 
ic

In [ ]:
df= pd.DataFrame(lines)
dataset2=df
name=['Entreprise', 'Annee', 'TCPP', 'CapitalPropre', 'PassifNonC', 'TActif',
       'TPassif', 'PassifCourant', 'ActifNonC', 'ActifCourant', 'Reserves',
       'CapitalSociale']
df

,Entreprise,Annee,TCPP,CapitalPropre,PassifNonC,TActif,TPassif,PassifCourant,ActifNonC,ActifCourant,...,TChargeExploitation,R_Exploitation,Res_Exsercice,ActiviteOrdinaire,FT_Exploitation,FT_Investisement,FT_Financement,TDebut,TFin,Variation
0,ab,2017,8 656 198,818 957,,8 656 198,7 837 241,,,,...,(356 866),127 809,113 911,115 905,(160 862),(134 468),24 349,(693 532),(964 513),(270 981)
1,ab,2018,8 885 599,978 601,,8 885 599,7 902 909,,,,...,(450 716),140 547,125 280,127 517,686 924,(198 104),(120 359),(961 189),(593 794),367 395
2,ab,2019,9 011 329,1 081 664,,9 011 329,7 925 654,,,,...,(498 415),155 860,146 556,147 746,340 501,126 945,(18 446),(593 794),(149 081),444 713
3,adwya,2016,85 735 875,34 183 160,8 413 653,85 735 875,51 552 715,43 139 062,30 986 623,54 749 253,...,(2 974 324),6 321 887,2 734 405,2 897 574,10 091 073,(4 625 548),(5 843 513),917 634,539 647,(377 988)
4,adwya,2018,112 132 832,39 002 918,13 751 706,112 132 832,73 129 914,59 378 208,39 316 054,72 816 778,...,(7 386 136),11 019 795,3 122 741,3 122 741,(5 058 101),(5 678 434),9 898 600,252 814,(585 121),(837 935)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,unimed,2017,101 764 019,82 622 654,4 956 516,101 764 019,19 141 365,14 184 849,48 894 637,52 869 382,...,763 560,15 013 896,12 532 575,12 532 575,17 495 962,<13 691 472>,<5 671 350>,12 694 052,10 827 192,<1 866 860>
73,unimed,2018,132 087 929,,16 911 489,132 087 929,38 648 360,21 736 871,62 177 016,69 910 913,...,(749 934),23 894 584,18 359 678,,13 380 031,(18 156 649),1 946 671,10 827 192,7 997 245,(2 829 947)
74,unimed,2019,149 607 872,,27 204 327,149 607 872,49 521 202,22 316 875,75 568 504,74 039 368,...,<5 658 094>,28 816 067,17 692 251,17 692 251,14 582 125,<18 364 430>,,7 997 245,3 170 677,<4 826 568>
75,wifack,2017,401 852 577,170 687 632,,401 852 577,231 164 945,,,,...,(3 364 046),(1 636 144),(1 653 156),(1 648 156),59 277 104,(39 968 767),(23 283 991),37 244 137,33 268 483,(3 975 654)


In [ ]:
dataset2.to_csv('dataset.csv',index=False)


###### Entreprise
Nom entreprise
######  Annee
Année raport annuel
###### TCPP
Total capitaux propres passifs
###### CapitalPropre
Passif Courant
###### PassifNonC
Passif non courant
###### TActif
Total actifs
###### TPassif
Total Passifs
###### PassifCourant
Passif Courant
###### TProdExpl
Total produits d'exploitation
###### Produit_Net 
produit net bancaire
###### TChargeExploitation
charges d'exploitation
###### R_Exploitation
Résultat d'exploitation
###### Res_Exsercice
Résultat de l'exercice
###### ActiviteOrdinaire
Activités ordinaires
###### FT_Exploitation
flux de trésorerie d'exploitation
###### FT_Investisement
flux de trésorerie d'investissement
######  FT_Financement
flux de trésorerie provenant des activités de financement
###### TDeb
liquidités et équivalents de liquidités début de période
trésorerie au début de l'exercice
###### TFin
liquidités et équivalents de liquidités fin de période
trésorerie à la clôture de l'exercice
###### Variation
variation nette des liquidités
variation de trésorerie


### Extracted Data Preparation

#### Données Externes : Chiffre d'affaire

In [ ]:
chiffre = pd.read_csv("Chiffre.csv",sep=";")
chiffre

,Entreprise,Annee,Chiffre
0,ab,2017,716 968
1,ab,2018,829 094
2,ab,2019,908 101
3,adwya,2016,84 750
4,adwya,2018,108 179
...,...,...,...
72,unimed,2017,70 345
73,unimed,2018,87 088
74,unimed,2019,107 206
75,wifack,2017,15 891


In [ ]:
dataset2=dataset2.assign(Chiffre_Affaire=chiffre["Chiffre"])

In [ ]:
dataset2.to_csv('datasetCombinée.csv',index=False)

In [ ]:
import pandas as pd
dataset = pd.read_csv('datasetCombinée.csv')

In [ ]:
dataset

,Entreprise,Annee,TCPP,CapitalPropre,PassifNonC,TActif,TPassif,PassifCourant,ActifNonC,ActifCourant,...,R_Exploitation,Res_Exsercice,ActiviteOrdinaire,FT_Exploitation,FT_Investisement,FT_Financement,TDebut,TFin,Variation,Chiffre_Affaire
0,ab,2017,8 656 198,818 957,NaN,8 656 198,7 837 241,NaN,NaN,NaN,...,127 809,113 911,115 905,(160 862),(134 468),24 349,(693 532),(964 513),(270 981),716 968
1,ab,2018,8 885 599,978 601,NaN,8 885 599,7 902 909,NaN,NaN,NaN,...,140 547,125 280,127 517,686 924,(198 104),(120 359),(961 189),(593 794),367 395,829 094
2,ab,2019,9 011 329,1 081 664,NaN,9 011 329,7 925 654,NaN,NaN,NaN,...,155 860,146 556,147 746,340 501,126 945,(18 446),(593 794),(149 081),444 713,908 101
3,adwya,2016,85 735 875,34 183 160,8 413 653,85 735 875,51 552 715,43 139 062,30 986 623,54 749 253,...,6 321 887,2 734 405,2 897 574,10 091 073,(4 625 548),(5 843 513),917 634,539 647,(377 988),84 750
4,adwya,2018,112 132 832,39 002 918,13 751 706,112 132 832,73 129 914,59 378 208,39 316 054,72 816 778,...,11 019 795,3 122 741,3 122 741,(5 058 101),(5 678 434),9 898 600,252 814,(585 121),(837 935),108 179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,unimed,2017,101 764 019,82 622 654,4 956 516,101 764 019,19 141 365,14 184 849,48 894 637,52 869 382,...,15 013 896,12 532 575,12 532 575,17 495 962,<13 691 472>,<5 671 350>,12 694 052,10 827 192,<1 866 860>,70 345
73,unimed,2018,132 087 929,NaN,16 911 489,132 087 929,38 648 360,21 736 871,62 177 016,69 910 913,...,23 894 584,18 359 678,NaN,13 380 031,(18 156 649),1 946 671,10 827 192,7 997 245,(2 829 947),87 088
74,unimed,2019,149 607 872,NaN,27 204 327,149 607 872,49 521 202,22 316 875,75 568 504,74 039 368,...,28 816 067,17 692 251,17 692 251,14 582 125,<18 364 430>,NaN,7 997 245,3 170 677,<4 826 568>,107 206
75,wifack,2017,401 852 577,170 687 632,NaN,401 852 577,231 164 945,NaN,NaN,NaN,...,(1 636 144),(1 653 156),(1 648 156),59 277 104,(39 968 767),(23 283 991),37 244 137,33 268 483,(3 975 654),15 891


In [ ]:
dataset.replace(to_replace ='\(', value = '-', regex = True,inplace=True)
dataset.replace(to_replace ='\<', value = '-', regex = True,inplace=True)
dataset.replace(to_replace ='\)', value = '', regex = True,inplace=True)
dataset.replace(to_replace ='\>', value = '', regex = True,inplace=True)
dataset.replace(to_replace ='\s', value = '', regex = True,inplace=True)
dataset.shape
dataset

,Entreprise,Annee,TCPP,CapitalPropre,PassifNonC,TActif,TPassif,PassifCourant,ActifNonC,ActifCourant,...,R_Exploitation,Res_Exsercice,ActiviteOrdinaire,FT_Exploitation,FT_Investisement,FT_Financement,TDebut,TFin,Variation,Chiffre_Affaire
0,ab,2017,8656198,818957,NaN,8656198,7837241,NaN,NaN,NaN,...,127809,113911,115905,-160862,-134468,24349,-693532,-964513,-270981,716968
1,ab,2018,8885599,978601,NaN,8885599,7902909,NaN,NaN,NaN,...,140547,125280,127517,686924,-198104,-120359,-961189,-593794,367395,829094
2,ab,2019,9011329,1081664,NaN,9011329,7925654,NaN,NaN,NaN,...,155860,146556,147746,340501,126945,-18446,-593794,-149081,444713,908101
3,adwya,2016,85735875,34183160,8413653,85735875,51552715,43139062,30986623,54749253,...,6321887,2734405,2897574,10091073,-4625548,-5843513,917634,539647,-377988,84750
4,adwya,2018,112132832,39002918,13751706,112132832,73129914,59378208,39316054,72816778,...,11019795,3122741,3122741,-5058101,-5678434,9898600,252814,-585121,-837935,108179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,unimed,2017,101764019,82622654,4956516,101764019,19141365,14184849,48894637,52869382,...,15013896,12532575,12532575,17495962,-13691472,-5671350,12694052,10827192,-1866860,70345
73,unimed,2018,132087929,NaN,16911489,132087929,38648360,21736871,62177016,69910913,...,23894584,18359678,NaN,13380031,-18156649,1946671,10827192,7997245,-2829947,87088
74,unimed,2019,149607872,NaN,27204327,149607872,49521202,22316875,75568504,74039368,...,28816067,17692251,17692251,14582125,-18364430,NaN,7997245,3170677,-4826568,107206
75,wifack,2017,401852577,170687632,NaN,401852577,231164945,NaN,NaN,NaN,...,-1636144,-1653156,-1648156,59277104,-39968767,-23283991,37244137,33268483,-3975654,15891


Types of the data

In [ ]:
print(dataset.dtypes)

Entreprise             object
Annee                  object
TCPP                   object
CapitalPropre          object
PassifNonC             object
TActif                 object
TPassif                object
PassifCourant          object
ActifNonC              object
ActifCourant           object
Reserves               object
CapitalSociale         object
TProdExploitation      object
TChargeExploitation    object
R_Exploitation         object
Res_Exsercice          object
ActiviteOrdinaire      object
FT_Exploitation        object
FT_Investisement       object
FT_Financement         object
TDebut                 object
TFin                   object
Variation              object
Chiffre_Affaire        object
dtype: object


In [ ]:
typ_objet =dataset.columns


In [ ]:
typ=typ_objet[2:]

In [ ]:

for index, row in dataset.iterrows():
    for i in range(len(typ)):
        CapitalPropre = dataset.columns.get_loc(typ[i])
        new_col = pd.to_numeric(dataset.iloc[:,CapitalPropre], errors='coerce')
        #coerce: the invalid parsing will be set as NaN
        dataset.iloc[:, CapitalPropre] = pd.Series(new_col)


In [ ]:
print(dataset.dtypes)

Entreprise              object
Annee                   object
TCPP                   float64
CapitalPropre          float64
PassifNonC             float64
TActif                 float64
TPassif                float64
PassifCourant          float64
ActifNonC              float64
ActifCourant           float64
Reserves               float64
CapitalSociale         float64
TProdExploitation      float64
TChargeExploitation    float64
R_Exploitation         float64
Res_Exsercice          float64
ActiviteOrdinaire      float64
FT_Exploitation        float64
FT_Investisement       float64
FT_Financement         float64
TDebut                 float64
TFin                   float64
Variation              float64
Chiffre_Affaire          int64
dtype: object


In [ ]:
dataset

,Entreprise,Annee,TCPP,CapitalPropre,PassifNonC,TActif,TPassif,PassifCourant,ActifNonC,ActifCourant,...,R_Exploitation,Res_Exsercice,ActiviteOrdinaire,FT_Exploitation,FT_Investisement,FT_Financement,TDebut,TFin,Variation,Chiffre_Affaire
0,ab,2017,8656198.0,818957.0,NaN,8656198.0,7837241.0,NaN,NaN,NaN,...,127809.0,113911.0,115905.0,-160862.0,-134468.0,24349.0,-693532.0,-964513.0,-270981.0,716968
1,ab,2018,8885599.0,978601.0,NaN,8885599.0,7902909.0,NaN,NaN,NaN,...,140547.0,125280.0,127517.0,686924.0,-198104.0,-120359.0,-961189.0,-593794.0,367395.0,829094
2,ab,2019,9011329.0,1081664.0,NaN,9011329.0,7925654.0,NaN,NaN,NaN,...,155860.0,146556.0,147746.0,340501.0,126945.0,-18446.0,-593794.0,-149081.0,444713.0,908101
3,adwya,2016,85735875.0,34183160.0,8413653.0,85735875.0,51552715.0,43139062.0,30986623.0,54749253.0,...,6321887.0,2734405.0,2897574.0,10091073.0,-4625548.0,-5843513.0,917634.0,539647.0,-377988.0,84750
4,adwya,2018,112132832.0,39002918.0,13751706.0,112132832.0,73129914.0,59378208.0,39316054.0,72816778.0,...,11019795.0,3122741.0,3122741.0,-5058101.0,-5678434.0,9898600.0,252814.0,-585121.0,-837935.0,108179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,unimed,2017,101764019.0,82622654.0,4956516.0,101764019.0,19141365.0,14184849.0,48894637.0,52869382.0,...,15013896.0,12532575.0,12532575.0,17495962.0,-13691472.0,-5671350.0,12694052.0,10827192.0,-1866860.0,70345
73,unimed,2018,132087929.0,NaN,16911489.0,132087929.0,38648360.0,21736871.0,62177016.0,69910913.0,...,23894584.0,18359678.0,NaN,13380031.0,-18156649.0,1946671.0,10827192.0,7997245.0,-2829947.0,87088
74,unimed,2019,149607872.0,NaN,27204327.0,149607872.0,49521202.0,22316875.0,75568504.0,74039368.0,...,28816067.0,17692251.0,17692251.0,14582125.0,-18364430.0,NaN,7997245.0,3170677.0,-4826568.0,107206
75,wifack,2017,401852577.0,170687632.0,NaN,401852577.0,231164945.0,NaN,NaN,NaN,...,-1636144.0,-1653156.0,-1648156.0,59277104.0,-39968767.0,-23283991.0,37244137.0,33268483.0,-3975654.0,15891


### Replacing the missing values with the appropriate formules

In [ ]:
#verifier les valeurs vides
print(dataset.isna().sum())

Entreprise              0
Annee                   0
TCPP                    6
CapitalPropre           9
PassifNonC             26
TActif                  2
TPassif                 5
PassifCourant          21
ActifNonC              21
ActifCourant           21
Reserves                1
CapitalSociale         11
TProdExploitation       3
TChargeExploitation     2
R_Exploitation          4
Res_Exsercice           7
ActiviteOrdinaire      12
FT_Exploitation        15
FT_Investisement       19
FT_Financement         23
TDebut                  2
TFin                    6
Variation               4
Chiffre_Affaire         0
dtype: int64


In [ ]:
#formule de remplacement du capital propre
dataset.loc[dataset.TCPP.isna(),"TCPP"]=dataset["TActif"]
dataset.loc[dataset.TCPP.isna(),"TCPP"]=dataset["CapitalPropre"]-dataset["TPassif"]
dataset.loc[dataset.CapitalPropre.isna(),"CapitalPropre"] = dataset["TCPP"]-dataset["TPassif"]
dataset.loc[dataset.TPassif.isnull(),"TPassif"]=dataset["TCPP"]-dataset["CapitalPropre"]
#formule de remplacement des passifs dans le bilan
dataset.loc[dataset.PassifNonC.isnull(),"PassifNonC"]=dataset["TPassif"]-dataset["PassifCourant"]
dataset.loc[dataset.TPassif.isnull(),"TPassif"]=dataset["PassifNonC"]+dataset["PassifCourant"]
dataset.loc[dataset.PassifCourant.isnull(),"PassifCourant"]=dataset["TPassif"]-dataset["PassifNonC"]
dataset.loc[dataset.TActif.isna(),"TActif"] = dataset["TCPP"]
dataset.loc[dataset.ActifCourant.isna(),"ActifCourant"] = dataset["TActif"]/2
dataset.loc[dataset.ActifNonC.isna(),"ActifNonC"] = dataset["TActif"]/2
dataset.loc[dataset.PassifNonC.isna(),"PassifCourant"] = dataset["TPassif"]/2
dataset.loc[dataset.PassifNonC.isna(),"PassifNonC"] = dataset["TPassif"]/2
dataset.loc[dataset.ActifCourant.isna(),"PassifCourant"] = dataset["TPassif"]/2
dataset.loc[dataset.ActiviteOrdinaire.isna(),"ActiviteOrdinaire"] = dataset["Res_Exsercice"]
#Resultat de l'exercice
dataset.loc[dataset.Res_Exsercice.isnull(),"Res_Exsercice"]=dataset["TActif"]-dataset["TPassif"]
#formule de remplace des exploitation dans l'etat de resultat
dataset.loc[dataset.R_Exploitation.isnull(),"R_Exploitation"]=dataset["TProdExploitation"]-dataset["TChargeExploitation"]
dataset.loc[dataset.TProdExploitation.isnull(),"TProdExploitation"]=dataset["R_Exploitation"]+dataset["TChargeExploitation"]
dataset.loc[dataset.TChargeExploitation.isnull(),"TChargeExploitation"]=dataset["TProdExploitation"]-dataset["R_Exploitation"]
dataset.loc[dataset.Variation.isna(),"Variation"] = dataset["TFin"]-dataset["TDebut"]
dataset.loc[dataset.TFin.isna(),"TFin"] = dataset["Variation"]-dataset["TDebut"]
dataset.loc[dataset.TDebut.isna(),"TDebut"] = dataset["Variation"]-dataset["TFin"]
dataset.loc[dataset.TFin.isna(),"TFin"] = dataset["Variation"]-dataset["TDebut"]
dataset.loc[dataset.Reserves.isnull(),"Reserves"]=dataset["CapitalPropre"]-dataset["CapitalSociale"]-dataset["Res_Exsercice"]
dataset.loc[dataset.CapitalSociale.isna(),"CapitalSociale"]=dataset["CapitalPropre"]-dataset["Reserves"]-dataset["Res_Exsercice"]


In [ ]:
print(dataset.isna().sum())

Entreprise              0
Annee                   0
TCPP                    0
CapitalPropre           0
PassifNonC              0
TActif                  0
TPassif                 0
PassifCourant           0
ActifNonC               0
ActifCourant            0
Reserves                1
CapitalSociale          1
TProdExploitation       2
TChargeExploitation     2
R_Exploitation          1
Res_Exsercice           0
ActiviteOrdinaire       3
FT_Exploitation        15
FT_Investisement       19
FT_Financement         23
TDebut                  1
TFin                    3
Variation               2
Chiffre_Affaire         0
dtype: int64


In [ ]:
dataset.head(60)

,Entreprise,Annee,TCPP,CapitalPropre,PassifNonC,TActif,TPassif,PassifCourant,ActifNonC,ActifCourant,...,R_Exploitation,Res_Exsercice,ActiviteOrdinaire,FT_Exploitation,FT_Investisement,FT_Financement,TDebut,TFin,Variation,Chiffre_Affaire
0,ab,2017,8.656198e+06,8.189570e+05,3.918620e+06,8.656198e+06,7.837241e+06,3.918620e+06,4.328099e+06,4.328099e+06,...,127809.0,113911.0,1.159050e+05,-160862.0,-134468.0,24349.0,-693532.0,-964513.0,-270981.0,716968
1,ab,2018,8.885599e+06,9.786010e+05,3.951454e+06,8.885599e+06,7.902909e+06,3.951454e+06,4.442800e+06,4.442800e+06,...,140547.0,125280.0,1.275170e+05,686924.0,-198104.0,-120359.0,-961189.0,-593794.0,367395.0,829094
2,ab,2019,9.011329e+06,1.081664e+06,3.962827e+06,9.011329e+06,7.925654e+06,3.962827e+06,4.505664e+06,4.505664e+06,...,155860.0,146556.0,1.477460e+05,340501.0,126945.0,-18446.0,-593794.0,-149081.0,444713.0,908101
3,adwya,2016,8.573588e+07,3.418316e+07,8.413653e+06,8.573588e+07,5.155272e+07,4.313906e+07,3.098662e+07,5.474925e+07,...,6321887.0,2734405.0,2.897574e+06,10091073.0,-4625548.0,-5843513.0,917634.0,539647.0,-377988.0,84750
4,adwya,2018,1.121328e+08,3.900292e+07,1.375171e+07,1.121328e+08,7.312991e+07,5.937821e+07,3.931605e+07,7.281678e+07,...,11019795.0,3122741.0,3.122741e+06,-5058101.0,-5678434.0,9898600.0,252814.0,-585121.0,-837935.0,108179
5,adwya,2019,1.141900e+08,3.833618e+07,1.216640e+07,1.141900e+08,7.585384e+07,6.368744e+07,4.247207e+07,7.171795e+07,...,5580297.0,1547713.0,1.547713e+06,8248989.0,-3706492.0,-6936927.0,-585121.0,-2979551.0,-2394430.0,106194
6,aetec,2018,7.688829e+06,-4.859526e+06,1.577820e+05,7.688829e+06,1.254835e+07,1.239057e+07,5.994780e+05,7.089350e+06,...,-1674724.0,-2287178.0,-2.287178e+06,2223956.0,40911.0,-751566.0,-3141014.0,-1627712.0,1513301.0,9980
7,alkim,2018,1.261277e+08,8.206197e+06,5.806015e+06,1.261277e+08,1.179215e+08,1.121155e+08,3.937832e+07,8.674934e+07,...,-24828036.0,-21836518.0,-2.183652e+07,15456455.0,-4261278.0,-9439027.0,2204695.0,4210688.0,2005992.0,116970
8,alkim,2017,1.896022e+08,3.203151e+07,4.113197e+07,1.896022e+08,1.575707e+08,1.164383e+08,8.297612e+07,1.066261e+08,...,-15953905.0,32031513.0,NaN,22688464.0,-2116427.0,-17546586.0,5432649.0,8458100.0,3025451.0,150684
9,alkim,2019,2.361764e+08,-1.567173e+07,5.354316e+07,2.361764e+08,2.518481e+08,1.983049e+08,1.180882e+08,1.180882e+08,...,-19094786.0,-15671729.0,NaN,NaN,NaN,NaN,4995043.0,23252154.0,18257111.0,176909


In [ ]:
i=0
for l in dataset.apply(lambda x: x['TCPP'] - x['TActif'], axis = 1):
    print(i,' ',l)
    i=i+1

0   0.0
1   0.0
2   0.0
3   0.0
4   0.0
5   0.0
6   0.0
7   0.0
8   0.0
9   0.0
10   0.0
11   0.0
12   0.0
13   0.0
14   0.0
15   0.0
16   0.0
17   0.0
18   0.0
19   0.0
20   0.0
21   0.0
22   0.0
23   0.0
24   0.0
25   0.0
26   0.0
27   0.0
28   0.0
29   0.0
30   0.0
31   -1.0
32   0.0
33   0.0
34   0.0
35   0.0
36   0.0
37   0.0
38   0.0
39   0.0
40   0.0
41   0.0
42   0.0
43   0.0
44   0.0
45   0.0
46   0.0
47   0.0
48   0.0
49   0.0
50   0.0
51   0.0
52   0.0
53   0.0
54   0.0
55   0.0
56   0.0
57   0.0
58   0.0
59   0.0
60   0.0
61   0.0
62   0.0
63   0.0
64   0.0
65   0.0
66   0.0
67   0.0
68   0.0
69   0.0
70   0.0
71   0.0
72   0.0
73   0.0
74   0.0
75   0.0
76   0.0


In [ ]:
dataset

,Entreprise,Annee,TCPP,CapitalPropre,PassifNonC,TActif,TPassif,PassifCourant,ActifNonC,ActifCourant,...,R_Exploitation,Res_Exsercice,ActiviteOrdinaire,FT_Exploitation,FT_Investisement,FT_Financement,TDebut,TFin,Variation,Chiffre_Affaire
0,ab,2017,8656198.0,818957.0,3918620.5,8656198.0,7837241.0,3918620.5,4328099.0,4328099.0,...,127809.0,113911.0,115905.0,-160862.0,-134468.0,24349.0,-693532.0,-964513.0,-270981.0,716968
1,ab,2018,8885599.0,978601.0,3951454.5,8885599.0,7902909.0,3951454.5,4442799.5,4442799.5,...,140547.0,125280.0,127517.0,686924.0,-198104.0,-120359.0,-961189.0,-593794.0,367395.0,829094
2,ab,2019,9011329.0,1081664.0,3962827.0,9011329.0,7925654.0,3962827.0,4505664.5,4505664.5,...,155860.0,146556.0,147746.0,340501.0,126945.0,-18446.0,-593794.0,-149081.0,444713.0,908101
3,adwya,2016,85735875.0,34183160.0,8413653.0,85735875.0,51552715.0,43139062.0,30986623.0,54749253.0,...,6321887.0,2734405.0,2897574.0,10091073.0,-4625548.0,-5843513.0,917634.0,539647.0,-377988.0,84750
4,adwya,2018,112132832.0,39002918.0,13751706.0,112132832.0,73129914.0,59378208.0,39316054.0,72816778.0,...,11019795.0,3122741.0,3122741.0,-5058101.0,-5678434.0,9898600.0,252814.0,-585121.0,-837935.0,108179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,unimed,2017,101764019.0,82622654.0,4956516.0,101764019.0,19141365.0,14184849.0,48894637.0,52869382.0,...,15013896.0,12532575.0,12532575.0,17495962.0,-13691472.0,-5671350.0,12694052.0,10827192.0,-1866860.0,70345
73,unimed,2018,132087929.0,93439569.0,16911489.0,132087929.0,38648360.0,21736871.0,62177016.0,69910913.0,...,23894584.0,18359678.0,18359678.0,13380031.0,-18156649.0,1946671.0,10827192.0,7997245.0,-2829947.0,87088
74,unimed,2019,149607872.0,100086670.0,27204327.0,149607872.0,49521202.0,22316875.0,75568504.0,74039368.0,...,28816067.0,17692251.0,17692251.0,14582125.0,-18364430.0,NaN,7997245.0,3170677.0,-4826568.0,107206
75,wifack,2017,401852577.0,170687632.0,115582472.5,401852577.0,231164945.0,115582472.5,200926288.5,200926288.5,...,-1636144.0,-1653156.0,-1648156.0,59277104.0,-39968767.0,-23283991.0,37244137.0,33268483.0,-3975654.0,15891
